## Import given data

In [167]:
import pandas as pd
import numpy as np
import json

In [168]:
## import initial data
# filtrato = pd.read_csv('data/storico_filtrato.csv', decimal=',')
# ordini = pd.read_csv('data/storico_ordini.csv')
# pianificazione = pd.read_csv('data/storico_pianificazione.csv')


## Cleaning

### lista_articoli.csv

cod_articolo x caratteristiche

In [169]:
articoli = pd.read_csv('data//input/lista_articoli.csv', decimal=',')      # add dtype={'programma ribobinatura S': np.int64}?

articoli.info()
articoli.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 616 entries, 0 to 615
Data columns (total 20 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   codarticolo               616 non-null    object 
 1   descrizione               616 non-null    object 
 2   anima                     616 non-null    object 
 3   copertura                 616 non-null    object 
 4   spire_reali               616 non-null    int64  
 5   velocita calcolata        616 non-null    int64  
 6   kg_ora                    616 non-null    float64
 7   spandex                   616 non-null    float64
 8   ciclo                     616 non-null    object 
 9   no_cicli                  616 non-null    int64  
 10  ore_levata                616 non-null    float64
 11  Divisore                  57 non-null     float64
 12  peso elastomero           558 non-null    float64
 13  Peso spola                580 non-null    float64
 14  programma 

,codarticolo,descrizione,anima,copertura,spire_reali,velocita calcolata,kg_ora,spandex,ciclo,no_cicli,ore_levata,Divisore,peso elastomero,Peso spola,programma ribobinatura S,programma ribobinatura Z,programma termofissaggio,kw tempo,kw percentuale,fine validita
0,10353ZF,"Ly 78/44f13 Z tpm 1100 st. 2,6 (CG)",Lycra 78 162C,44f13 test. S PA 66,1050,17200,1.87,2.76,2 x 4 gg (192),2,96.0,NaN,530.0,435.0,NaN,25,4.0,NaN,NaN,NaN
1,10369ZF/2013,"Cr 44/78f18 Z tpm 600 st. 3,7 (DH)",Creora 44 H350,78f18 Z PA 6 vari colori,600,13200,3.15,1.49,5 x 3 gg + 4 h (378),5,75.6,NaN,550.0,803.0,NaN,41,6.0,NaN,NaN,NaN
2,10369ZF2,"Cr 44/78f18 Z tpm 600 st. 3,7 (DH)",Creora 44 H350,78f18 Z PA 6 francia e nero,600,13200,3.15,1.49,5 x 3 gg + 4 h (378),5,75.6,NaN,550.0,803.0,NaN,41,11.0,NaN,NaN,NaN
3,10985ZF,"Ly44/78f68 Z tpm 1000 st. 3,3 (DD)",Lycra 44 162C,78f68 test. S PA 66,960,13500,2.05,1.10,4 x 4 gg + 22 h (472),4,118.0,NaN,517.0,820.0,NaN,41,11.0,NaN,NaN,NaN
4,10985ZF spole 880 gr,"Ly44/78f68 Z tpm 1000 st. 3,3 (DD)",Lycra 44 162C,78f68 test. S PA 66,960,12300,1.87,1.00,3 x 5 gg + 19 h (417) 420 gr lycra,3,139.0,NaN,517.0,880.0,NaN,41,11.0,NaN,NaN,NaN


#### Controllo totale ore

In [170]:
df = articoli.copy()

## add totale ore
df = df.loc[:, ["codarticolo", "ciclo", "no_cicli", "ore_levata"]]

# check ore
df['totale_ore descritto'] = df['ciclo'].str.extract(r'\(( *\d+(?:,\d+)*)\)')
df['totale_ore descritto'] = df['totale_ore descritto'].str.replace(',', '.').astype(float)

df['totale_ore'] = df['no_cicli'] * df['ore_levata']
df['verifica ore'] = np.equal(df['no_cicli'] * df['ore_levata'], df['totale_ore descritto'])
df['differenza ore'] = np.sqrt((df['totale_ore'] - df['totale_ore descritto'].where(df['totale_ore descritto'].notna(), 0).astype(int))**2)

average_percentage = (df['verifica ore'].sum() / df['verifica ore'].count()) * 100
print(f"Totale ore in Ciclo e totale ore calcolate con ore_levata * no_cicli combaciano nel: \n{average_percentage:.2f}%")

df.head()

Totale ore in Ciclo e totale ore calcolate con ore_levata * no_cicli combaciano nel: 
56.66%


,codarticolo,ciclo,no_cicli,ore_levata,totale_ore descritto,totale_ore,verifica ore,differenza ore
0,10353ZF,2 x 4 gg (192),2,96.0,192.0,192.0,True,0.0
1,10369ZF/2013,5 x 3 gg + 4 h (378),5,75.6,378.0,378.0,True,0.0
2,10369ZF2,5 x 3 gg + 4 h (378),5,75.6,378.0,378.0,True,0.0
3,10985ZF,4 x 4 gg + 22 h (472),4,118.0,472.0,472.0,True,0.0
4,10985ZF spole 880 gr,3 x 5 gg + 19 h (417) 420 gr lycra,3,139.0,417.0,417.0,True,0.0


Per visualizzare le righe ambigue modificare epsilon (tolleranza):

In [171]:
epsilon = 100      # minimo differenza
ambiguo = df[df['verifica ore'] == False].where((np.sqrt(df['differenza ore'] ** 2)) > epsilon).dropna(subset=['codarticolo'])
print(f"Totale righe: {df.shape[0]}")
print(f"Righe ambigue: {ambiguo.shape[0]}")
ambiguo

Totale righe: 616
Righe ambigue: 17


,codarticolo,ciclo,no_cicli,ore_levata,totale_ore descritto,totale_ore,verifica ore,differenza ore
30,15613DN1,1 x 18 gg (432) /2,2.0,429.00,432.0,858.00,False,426.00
44,1723ZN1,6 x 5 gg + 22 h (850),4.0,169.00,850.0,676.00,False,174.00
45,1723ZN2,6 x 5 gg + 22 h (850),4.0,169.00,850.0,676.00,False,174.00
74,2221ZN,10 x 3 gg (725),3.0,24.16,725.0,72.48,False,652.52
102,2254ZN-11400,4 x 6 gg + 17 h (464),4.0,161.00,464.0,644.00,False,180.00
210,4405S+ZN,2 x 14 gg + 10 h (692),1.0,346.00,692.0,346.00,False,346.00
211,4405SN,2 x 14 gg + 10 h (692),1.0,346.00,692.0,346.00,False,346.00
212,4405ZN,2 x 14 gg + 10 h (692),1.0,346.00,692.0,346.00,False,346.00
281,749DN,2 x 17 gg + 21 h (431),2.0,431.00,431.0,862.00,False,431.00
282,749DN mini,2 x 17 gg + 21 h (431),2.0,431.00,431.0,862.00,False,431.00


##### Controllo numero cicli

In [172]:
# check cicli
df['no_cicli descritto'] = df['ciclo'].str.extract(r'(\d{0,2})').astype(int)
df['differenza cicli'] = df['no_cicli'] - df['no_cicli descritto']
df['verifica cicli'] = np.equal(df['no_cicli descritto'], df['no_cicli'])

average = (df['verifica cicli'].sum() / df['verifica cicli'].count())*100

print(f"no_cicli combaciano nel {average:.2f}%")
df[df['verifica cicli'] == False]

no_cicli combaciano nel 94.81%


,codarticolo,ciclo,no_cicli,ore_levata,totale_ore descritto,totale_ore,verifica ore,differenza ore,no_cicli descritto,differenza cicli,verifica cicli
9,11007SF 3 cicli,2 x 16 gg (1154),3,385.00,1154.0,1155.00,False,1.00,2,1,False
11,11011ZF2,7 x 8 gg (1345),6,224.17,1345.0,1345.02,False,0.02,7,-1,False
28,15611DN,2 x 2 gg + 23 h (141) /2,4,35.25,141.0,141.00,True,0.00,2,2,False
30,15613DN1,1 x 18 gg (432) /2,2,429.00,432.0,858.00,False,426.00,1,1,False
44,1723ZN1,6 x 5 gg + 22 h (850),4,169.00,850.0,676.00,False,174.00,6,-2,False
45,1723ZN2,6 x 5 gg + 22 h (850),4,169.00,850.0,676.00,False,174.00,6,-2,False
59,20098DN,5 x 1 gg + 22 h (230),3,46.00,230.0,138.00,False,92.00,5,-2,False
60,20098DN-VARI COL,5 x 1 gg + 22 h (230),3,46.00,230.0,138.00,False,92.00,5,-2,False
74,2221ZN,10 x 3 gg (725),3,24.16,725.0,72.48,False,652.52,10,-7,False
117,2269ZF2,5 x 5 gg + 21 h (706),4,176.50,706.0,706.00,True,0.00,5,-1,False


In alcuni casi i cicli sono diversi ma le ore sono uguali:

In [173]:
ambiguo = df[(df['verifica cicli'] == False) & (df['verifica ore'] == True)]
ambiguo.loc[:, ['codarticolo',  'ciclo', 'no_cicli','no_cicli descritto', 'ore_levata', 'totale_ore', 'totale_ore descritto', 'verifica ore', 'verifica cicli']]

,codarticolo,ciclo,no_cicli,no_cicli descritto,ore_levata,totale_ore,totale_ore descritto,verifica ore,verifica cicli
28,15611DN,2 x 2 gg + 23 h (141) /2,4,2,35.25,141.0,141.0,True,False
117,2269ZF2,5 x 5 gg + 21 h (706),4,5,176.50,706.0,706.0,True,False
118,2269ZN/F2,5 x 5 gg + 21 h (706),4,5,176.50,706.0,706.0,True,False
119,2269ZN2,5 x 5 gg + 21 h (706),4,5,176.50,706.0,706.0,True,False
153,23502DN-150053,2 x 3 gg + 18 h (181) /2,4,2,45.25,181.0,181.0,True,False
154,23502DN-9025,2 x 3 gg + 18 h (181) /2,4,2,45.25,181.0,181.0,True,False
155,23502DN-T0001,2 x 3 gg + 18 h (181) /2,4,2,45.25,181.0,181.0,True,False
187,31002DN1,1 x 5 gg + 20 h (140) /2,2,1,70.00,140.0,140.0,True,False
264,62002DN,1 x 4 gg (96) /2,2,1,48.00,96.0,96.0,True,False
314,S15-60008V2DN2,2 x 1 gg + 19 h (43),1,2,43.00,43.0,43.0,True,False


Vedo cosa cambia usando 'no_cicli' (v1) o 'no_cicli descritto' (v2):

In [174]:
tolleranza = 0

df['totale_ore v2'] = df['no_cicli descritto'] * df['ore_levata']
df['differenza ore v2'] = np.sqrt((df['totale_ore descritto'] - df['totale_ore v2'])**2)

v1 = df.where(df['differenza ore'] > tolleranza).dropna(subset='codarticolo')
v2 = df.where(df['differenza ore v2'] > tolleranza).dropna(subset='codarticolo')

print(f"v1.len = {v1.shape[0]}")
print(f"v2.len = {v2.shape[0]}")

v1.len = 267
v2.len = 269


$\implies$ non cambia praticamente nulla, anzi peggiora

##### Dataset valido

In [175]:
ok = df[(df['verifica ore'] == True) & df['verifica cicli'] == True]

print(f"numero righe originarie: {df.shape[0]}")
print(f"numero righe non ambigue: {ok.shape[0]}")
print(f"percentuale dataset valido: {(ok.shape[0] / df.shape[0])*100:.2f}%")

numero righe originarie: 616
numero righe non ambigue: 334
percentuale dataset valido: 54.22%


#### Add totale kg

Per aggiungere totale kg abbiamo bisogno delle *ore_totale* veritiere $\implies$ per ora consideriamo solo dataset valido

In [176]:
valid = articoli.where(articoli['codarticolo'].isin(ok['codarticolo'])).dropna(subset='codarticolo')
print(f"n_righe: {valid.shape[0]}")

df = valid

valid.head()

n_righe: 335


,codarticolo,descrizione,anima,copertura,spire_reali,velocita calcolata,kg_ora,spandex,ciclo,no_cicli,ore_levata,Divisore,peso elastomero,Peso spola,programma ribobinatura S,programma ribobinatura Z,programma termofissaggio,kw tempo,kw percentuale,fine validita
0,10353ZF,"Ly 78/44f13 Z tpm 1100 st. 2,6 (CG)",Lycra 78 162C,44f13 test. S PA 66,1050.0,17200.0,1.87,2.76,2 x 4 gg (192),2.0,96.0,NaN,530.0,435.0,NaN,25,4.0,NaN,NaN,NaN
1,10369ZF/2013,"Cr 44/78f18 Z tpm 600 st. 3,7 (DH)",Creora 44 H350,78f18 Z PA 6 vari colori,600.0,13200.0,3.15,1.49,5 x 3 gg + 4 h (378),5.0,75.6,NaN,550.0,803.0,NaN,41,6.0,NaN,NaN,NaN
2,10369ZF2,"Cr 44/78f18 Z tpm 600 st. 3,7 (DH)",Creora 44 H350,78f18 Z PA 6 francia e nero,600.0,13200.0,3.15,1.49,5 x 3 gg + 4 h (378),5.0,75.6,NaN,550.0,803.0,NaN,41,11.0,NaN,NaN,NaN
3,10985ZF,"Ly44/78f68 Z tpm 1000 st. 3,3 (DD)",Lycra 44 162C,78f68 test. S PA 66,960.0,13500.0,2.05,1.10,4 x 4 gg + 22 h (472),4.0,118.0,NaN,517.0,820.0,NaN,41,11.0,NaN,NaN,NaN
4,10985ZF spole 880 gr,"Ly44/78f68 Z tpm 1000 st. 3,3 (DD)",Lycra 44 162C,78f68 test. S PA 66,960.0,12300.0,1.87,1.00,3 x 5 gg + 19 h (417) 420 gr lycra,3.0,139.0,NaN,517.0,880.0,NaN,41,11.0,NaN,NaN,NaN


In [177]:
## add totale kg
df['totale_ore'] = df['ore_levata'] * df['no_cicli']
df['totale_kg'] = df['totale_ore'] * df['kg_ora']
df.head()

,codarticolo,descrizione,anima,copertura,spire_reali,velocita calcolata,kg_ora,spandex,ciclo,no_cicli,...,peso elastomero,Peso spola,programma ribobinatura S,programma ribobinatura Z,programma termofissaggio,kw tempo,kw percentuale,fine validita,totale_ore,totale_kg
0,10353ZF,"Ly 78/44f13 Z tpm 1100 st. 2,6 (CG)",Lycra 78 162C,44f13 test. S PA 66,1050.0,17200.0,1.87,2.76,2 x 4 gg (192),2.0,...,530.0,435.0,NaN,25,4.0,NaN,NaN,NaN,192.0,359.04
1,10369ZF/2013,"Cr 44/78f18 Z tpm 600 st. 3,7 (DH)",Creora 44 H350,78f18 Z PA 6 vari colori,600.0,13200.0,3.15,1.49,5 x 3 gg + 4 h (378),5.0,...,550.0,803.0,NaN,41,6.0,NaN,NaN,NaN,378.0,1190.70
2,10369ZF2,"Cr 44/78f18 Z tpm 600 st. 3,7 (DH)",Creora 44 H350,78f18 Z PA 6 francia e nero,600.0,13200.0,3.15,1.49,5 x 3 gg + 4 h (378),5.0,...,550.0,803.0,NaN,41,11.0,NaN,NaN,NaN,378.0,1190.70
3,10985ZF,"Ly44/78f68 Z tpm 1000 st. 3,3 (DD)",Lycra 44 162C,78f68 test. S PA 66,960.0,13500.0,2.05,1.10,4 x 4 gg + 22 h (472),4.0,...,517.0,820.0,NaN,41,11.0,NaN,NaN,NaN,472.0,967.60
4,10985ZF spole 880 gr,"Ly44/78f68 Z tpm 1000 st. 3,3 (DD)",Lycra 44 162C,78f68 test. S PA 66,960.0,12300.0,1.87,1.00,3 x 5 gg + 19 h (417) 420 gr lycra,3.0,...,517.0,880.0,NaN,41,11.0,NaN,NaN,NaN,417.0,779.79


#### Add consumo energetico

In [178]:
## add consume energetico
print(f"Abbiamo dati del consumo energito per {df['kw tempo'].notna().sum()} righe su {valid.shape[0]} totali.")
df[df['kw percentuale'].notna()].head()

Abbiamo dati del consumo energito per 18 righe su 335 totali.


,codarticolo,descrizione,anima,copertura,spire_reali,velocita calcolata,kg_ora,spandex,ciclo,no_cicli,...,peso elastomero,Peso spola,programma ribobinatura S,programma ribobinatura Z,programma termofissaggio,kw tempo,kw percentuale,fine validita,totale_ore,totale_kg
17,13005ZF-600gr,"Ly130/33f20 Z tpm 900 st. 4,2 (EC)",Lycra 130 902C,33f20 parallelo PA 66,855.0,14500.0,1.69,3.07,2 x 4 gg (192),2.0,...,1046.0,370.0,NaN,26,11.0,05:00:00,1.0,NaN,192.0,324.48
22,13007ZN/F-14500,"Ly130/33f20 Z tpm 900 st. 4,2 (EC)",Lycra 130 902C,33f20 test. S PA 66,900.0,14500.0,1.65,3.07,3 x 4 gg + 18 h (342),3.0,...,1046.0,390.0,NaN,19,11.0,05:00:00,1.0,NaN,342.0,564.30
63,2011DN mini,"Ly 20/13f10 dc tpm 2400 st. 3,0 (DA)",Lycra 20 188C,13f10 parallelo PA 66 mispole,2340.0,21000.0,0.25,0.35,2 x 14 gg + 23 h (718),2.0,...,268.0,310.0,NaN,29,NaN,10:00:00,0.2,NaN,718.0,179.50
120,2270,"Ly156/33f10 Z tpm 900 st. 4,3 (ED)",Lycra 156 162C,33f10 test. S PA 66,900.0,16100.0,1.95,3.90,3 x 3 gg + 19 h (273),3.0,...,1063.0,338.0,NaN,36,9.0,10:00:00,0.5,NaN,273.0,532.35
121,2270S+ZN,"KL22/11f5 S+Z tpm 1900 st. 2,8 (CI)",Mobilon 22 KL,11f5 miracosmo PA 6,1810.0,21000.0,0.34,0.57,2 x 15gg + 20 h (760),2.0,...,435.0,300.0,17,36,NaN,10:00:00,0.5,NaN,760.0,258.40


#### Conclusione

Non è chiaro come procedere quando:
- in ciclo è presente **/2**
- totale ore calcolate e ore in *ciclo* non combaciano
- no_cicli e no_cicli in *ciclo* non combaciano
- totale ore combaciano ma no_cicli no
- *ciclo* non segue il patern: **no_cicli $\times$ n_giorni + n_ore**

Inoltre:
- consumo energetico presente solo su 18 righe tra le 335 valide.

In [179]:
valid.to_csv('data/valid/lista_articoli.csv', index=False)

### storico_filtrato.csv

### storico_ordini.csv

### storico_pianificazione.csv

In [3]:
# sistema storico_filtrato

df = filtrato.copy()

df.drop(columns='N° MC', inplace=True)
df.rename(columns={
    'Unnamed: 1': 'anno inizio',
    'Unnamed: 2': 'mese articolo',
    'Unnamed: 3': 'N_mc'
    }, 
    inplace=True
)
df.dropna(subset=['data ora partenza'], inplace=True)

df['anno inizio'] = df['data ora partenza'].apply(lambda x: x.split(' ')[0].split('/')[2])
df['mese articolo'] = df['data ora partenza'].apply(lambda x: x.split(' ')[0].split('/')[1])

filtrato = df.copy()
filtrato.head()

,anno inizio,mese articolo,N_mc,cod articolo,descrizione,N° fusi reali,velocita fusi inferiori,data ora partenza,data ora fine,data ora fine manuale,ore lavorate dalla partenza,KWh assorbiti,KW medi,produzione [kg],KWh/Kg,€/Kg,note,N° scheda
2,2024,04,1.0,2277DN mini,"Ly22/8f5 dc tpm 2100 st. 3,4 (DE)",NaN,20194.0,11/04/2024 14:20:33,29/04/2024 21:59:49,NaN,439.48,5921.32,13.4735,121.47,48.74,10.72,x la 7°levata vedere se mettere lycra 22 usata...,NaN
3,2024,04,2.0,2011DN mini,"Ly 20/13f10 dc tpm 2400 st. 3,0 (DA)",NaN,21112.0,03/04/2024 13:41:13,17/04/2024 04:36:25,NaN,326.78,5583.02,17.0850,135.82,41.11,9.04,NaN,NaN
4,2024,04,3.0,2011DN mini,"Ly 20/13f10 dc tpm 2400 st. 3,0 (DA)",448.0,21026.0,24/04/2024 12:34:52,08/05/2024 07:06:28,NaN,330.38,5914.74,17.9028,137.49,43.02,9.46,NaN,NaN
5,2024,04,4.0,2011DN mini,"Ly 20/13f10 dc tpm 2400 st. 3,0 (DA)",NaN,20922.0,19/04/2024 11:01:44,06/05/2024 19:42:55,NaN,416.55,7735.80,18.5711,173.13,44.68,9.83,NaN,NaN
6,2024,04,5.0,2271DN 20700 mini,"K22/8f5 dc tpm 3000 st. 2,8 (CI)",448.0,21077.0,26/04/2024 13:48:42,21/05/2024 15:32:12,NaN,601.68,9070.20,15.0748,152.78,59.37,13.06,\n\n,23/70404-M8


# Databases

## Macchine x Prodotti

In [54]:
# clean data
df = filtrato.loc[:, ['N_mc', 'cod articolo']]
df = df.rename(columns={'Unnamed: 3': 'N_mc'})
df = df.dropna(subset=['N_mc'])

# remove duplicates and sort by N_mc
unique_df = df.drop_duplicates(subset=['N_mc', 'cod articolo'])
macchine_df = unique_df.sort_values(by=['N_mc'])

# macchine_df.head()

## dictionary from df
df = macchine_df

macchine = {}
for key in df['N_mc'].unique():
    values = [v for v in df[df['N_mc'] == key].loc[:, 'cod articolo'].values]
    macchine[int(key)] = values

# save to json
with open('data/macchine.json', 'w') as f:
    json.dump(macchine, f, skipkeys=True, ensure_ascii=False, indent=4)

# load from json
with open('data/macchine.json', 'r') as f:
    macchine = json.load(f)

macchine


,N_mc,cod articolo
2,1.0,2277DN mini
3,2.0,2011DN mini
4,3.0,2011DN mini
5,4.0,2011DN mini
441,5.0,2011DN mini


## Prodotti x Macchine

In [57]:
df = filtrato.loc[:, ['N_mc', 'cod articolo']]
df.columns = ['N_mc', 'cod articolo']

# switch columns
df = df[['cod articolo', 'N_mc']]
df.dropna(inplace=True)
df.head()

articoli_df = df

articoli = {}
for key in df['cod articolo'].unique():
    values = [int(v) for v in df[df['cod articolo'] == key].loc[:, 'N_mc'].unique()]
    articoli[key] = values

with open('data/articoli.json', 'w') as f:
    json.dump(articoli, f, skipkeys=True, ensure_ascii=False, indent=4)

articoli = json.load(open('data/articoli.json'))
articoli

{'2277DN mini': [1, 7],
 '2011DN mini': [2, 3, 4, 5],
 '2271DN 20700 mini': [5, 8, 9, 10, 11, 12, 47, 48, 62, 63, 64, 65, 66, 61],
 '749DN mini': [6, 49, 50],
 'X22-1703DN minispole': [7],
 'X23-3399S+ZF': [13, 44],
 '2270s+z rpm 19800': [14],
 '2270S+ZN 19800': [15, 16, 17, 19, 20, 26],
 'X23-2299V5ZF': [18, 72],
 '7032S+ZN': [18],
 '2270s+z rpm 21000': [21],
 '2250S+ZN': [22],
 '31002DN1': [23],
 'X24-4401S+ZF2': [24],
 'X23-4493S+ZF': [24, 25],
 'X21-2291SF': [25],
 'X21-2291sf': [25],
 '2270S+ZN mini': [26],
 'X21-2099S+ZN mini': [27, 28, 33, 34],
 'X24-2001S+ZN': [28, 60],
 '13007ZN/F-14500': [29, 30, 32, 43],
 '28503ZN': [29],
 '2246ZN 10800': [31, 44],
 'X23-2280S+ZN': [35],
 'X20-2222DN2 mini': [36],
 '33002DN': [37, 41],
 's22-700011DN': [37],
 'S600A04DN': [37],
 'S600A03DN': [37],
 '33001DN2': [38, 41, 67],
 'X23-8397DN': [38],
 '31003DN220274': [39, 40],
 'X21-39599V7DN0332': [39],
 '62002DN': [39],
 '39502DN-14446': [39],
 '39502dn': [39],
 '62002DN2': [39],
 'X21-20099DN2

## Prodotti
Articoli: macchine, (gruppo_macchine), n_levate (= no_cicli), n_fusi (joint con macchine, check ambigui), tempo_levata, kg_levata (kg_ora x ore_levata)
- toglio ambigui

### Macchine x caratteristiche

In [28]:
groups_filename= "data/Gruppi_Macchine_per_articoli.json"

In [144]:
with open(groups_filename, 'r') as f:
    groups_file = json.load(f)

divisione = groups_file['Divisione per articoli prodotti']
divisione.keys()

# elimina sottogruppi
groups = {}

for key, group_dict in divisione.items():
    if key + 'a' in group_dict.keys():
        groups[key + 'a'] = group_dict[key + 'a']
    if key + 'b' in group_dict.keys():
        groups[key + 'b'] = group_dict[key + 'b']
    else:
        groups[key] = group_dict

print(groups.keys())

# crea dataframe macchine: caratteristiche
data = []

for classe, group in groups.items():
    
    n_fusi = group.pop('Numero Fusi')
    
    for mode, mode_dict in group.items():
        descrizione = mode_dict.pop('Descrizione')
        macchine = mode_dict.pop('Macchine')

        for m in macchine:
            data.append([m, classe, n_fusi, mode, descrizione])

df = pd.DataFrame(data, columns=['N_mc', 'classe', 'n_fusi', 'mode', 'descrizione'])
df.sort_values(by='N_mc', inplace=True)

# save to json
df.to_json('data/macchine.json', orient='records', indent=4)


dict_keys(['Gruppo1a', 'Gruppo1b', 'Gruppo2', 'Gruppo3a', 'Gruppo3b', 'Gruppo4a', 'Gruppo4b', 'Gruppo5a', 'Gruppo5b', 'Gruppo6'])


# Statistiche

In [67]:
filtrato.head()


,anno inizio,mese articolo,N_mc,cod articolo,descrizione,N° fusi reali,velocita fusi inferiori,data ora partenza,data ora fine,data ora fine manuale,ore lavorate dalla partenza,KWh assorbiti,KW medi,produzione [kg],KWh/Kg,€/Kg,note,N° scheda
2,2024,04,1.0,2277DN mini,"Ly22/8f5 dc tpm 2100 st. 3,4 (DE)",NaN,20194.0,11/04/2024 14:20:33,29/04/2024 21:59:49,NaN,439.48,5921.32,13.4735,121.47,48.74,10.72,x la 7°levata vedere se mettere lycra 22 usata...,NaN
3,2024,04,2.0,2011DN mini,"Ly 20/13f10 dc tpm 2400 st. 3,0 (DA)",NaN,21112.0,03/04/2024 13:41:13,17/04/2024 04:36:25,NaN,326.78,5583.02,17.0850,135.82,41.11,9.04,NaN,NaN
4,2024,04,3.0,2011DN mini,"Ly 20/13f10 dc tpm 2400 st. 3,0 (DA)",448.0,21026.0,24/04/2024 12:34:52,08/05/2024 07:06:28,NaN,330.38,5914.74,17.9028,137.49,43.02,9.46,NaN,NaN
5,2024,04,4.0,2011DN mini,"Ly 20/13f10 dc tpm 2400 st. 3,0 (DA)",NaN,20922.0,19/04/2024 11:01:44,06/05/2024 19:42:55,NaN,416.55,7735.80,18.5711,173.13,44.68,9.83,NaN,NaN
6,2024,04,5.0,2271DN 20700 mini,"K22/8f5 dc tpm 3000 st. 2,8 (CI)",448.0,21077.0,26/04/2024 13:48:42,21/05/2024 15:32:12,NaN,601.68,9070.20,15.0748,152.78,59.37,13.06,\n\n,23/70404-M8


In [68]:
filtrato.describe()

,N_mc,N° fusi reali,velocita fusi inferiori,ore lavorate dalla partenza,KWh assorbiti,KW medi,produzione [kg],KWh/Kg,€/Kg
count,651.000000,360.000000,647.000000,602.000000,602.000000,602.000000,624.000000,575.000000,575.000000
mean,41.476190,268.880556,15973.378671,163.433538,1927.399485,9.925470,129.325913,15.894991,3.495096
std,18.640253,71.303499,4055.193289,151.716547,2231.308768,3.917078,63.858465,20.232336,4.451992
min,1.000000,24.000000,7000.000000,4.980000,0.120000,0.000300,0.260000,0.000000,0.000000
25%,30.000000,256.000000,13195.500000,58.765000,454.525000,7.743075,92.330000,3.650000,0.805000
50%,40.000000,256.000000,14623.000000,109.335000,1027.235000,10.275650,123.740000,6.680000,1.470000
75%,56.500000,264.000000,19936.000000,228.420000,2723.320000,12.511100,171.005000,26.790000,5.895000
max,75.000000,448.000000,21189.000000,649.520000,11224.170000,19.356400,795.300000,245.500000,54.010000


In [ ]:
filtrato['']